In [ ]:
#!pip install splinter

In [1]:
# Import dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from bs4 import BeautifulSoup
# import pandas as pd
# import time

# # Initialize the WebDriver
# driver = webdriver.Chrome()  # Ensure you have ChromeDriver installed
# driver.get("https://ircc.canada.ca/english/newcomers/services/index.asp#table1")

# # Wait for the dropdown to be visible and select "all"
# wait = WebDriverWait(driver, 20)
# try:
#     dropdown = wait.until(EC.visibility_of_element_located((By.NAME, "table1_length")))
#     dropdown.click()
#     all_option = driver.find_element(By.XPATH, "//option[@value='-1']")
#     all_option.click()
#     print("Successfully selected 'all' option from the dropdown.")

#     # Allow some time for all the rows to load
#     time.sleep(5)

#     # Get the page source and parse it with BeautifulSoup
#     soup = BeautifulSoup(driver.page_source, "html.parser")
#     table = soup.find("table", {"id": "table1"})

#     # List to store all extracted data
#     all_page_data = []

#     # Find all rows (tr) in the table
#     rows = table.find_all("tr")

#     # Iterate through each row to extract data
#     for row in rows:
#         # Extract Organisation Name
#         org_name_element = row.find("a", string=True)  # Find the <a> tag containing the org name
#         org_name = org_name_element.text.strip() if org_name_element else None

#         # Extract Organisation Website
#         org_website_element = row.find("a", href=True)
#         org_website = org_website_element["href"] if org_website_element else None
        
#         # Extract all <a> tags with href in the row
#         org_address_elements = row.find_all("a", href=True)

#         # Extract the Google Maps link
#         org_googlemap_element = org_address_elements[1] if len(org_address_elements) > 1 else None
#         org_googlemap = org_googlemap_element["href"] if org_googlemap_element else None

#         # Extract Organisation Address
#         org_address_element = org_address_elements[1] if len(org_address_elements) > 1 else None  # Get the second <a> tag
#         org_address = org_address_element.text.strip() if org_address_element else None

#         # Extract Languages of Service
#         lang_element = row.find("td")  # Assume languages are in a <td> tag
#         if lang_element and "Languages of service:" in lang_element.text:
#             org_languages = lang_element.text.split("Languages of service:")[1].strip()
#         else:
#             org_languages = None

#         # Extract Services
#         services_elements = row.find_all("li", class_="nowrap")
#         org_services = [service.text.strip() for service in services_elements]
        
#         # Extract details from all <td> elements
#         all_td_text = []
#         td_elements = row.find_all("td")
#         for td in td_elements:
#             for a_tag in td.find_all("a"):
#                 a_tag.decompose()  # Remove <a> tags to avoid hyperlink text
#             cell_text = td.get_text(strip=True)
#             if cell_text:
#                 all_td_text.append(cell_text)
#         org_details = "/".join(all_td_text)

#         # Organize extracted data into a dictionary
#         item_data = {
#             "Organisation Name": org_name,
#             "Organisation Website": org_website,
#             "Organisation Address": org_address,
#             "Organisation Languages": org_languages,
#             "Organisation Services": org_services,
#             "Organisation Google Map": org_googlemap,
#             "Organisation Details": org_details,
#         }

#         # Append the data to the list
#         all_page_data.append(item_data)

#     # Convert the collected data into a DataFrame
#     df = pd.DataFrame(all_page_data)

# except Exception as e:
#     print(f"Error encountered: {e}")

# finally:
#     # Close the driver
#     driver.quit()


In [ ]:
df.to_csv("scraped_organisations.csv", index=False)

In [9]:
# Initialize Splinter Browser
with Browser('chrome') as browser:
    # Define base URL
    url = 'https://www.quebec.ca/en/immigration/partners'
    browser.visit(url)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    # List to store all extracted data
    all_page_data = []

    # Find all partner elements
    partners = soup.find_all('div', class_='partner-element')

    # Iterate through each partner to extract data
    for partner in partners:
        # Extract Organisation Name
        org_name_element = partner.find('p', class_='partner-name').find('a')
        org_name = org_name_element.text.strip() if org_name_element else None

        # Extract Phone Number
        phone_element = partner.find(text=lambda text: 'Tel.' in text if text else False)
        phone = phone_element.strip() if phone_element else None

        # Extract Languages
        language_element = partner.find('td', class_='langs')
        languages = language_element.text.strip() if language_element else None
        
        # Extract all <a> tags with href in the row
        org_address_elements = partner.find_all("a", href=True)
        # Find the Google Maps link based on URL pattern
        org_googlemap_element = next((a for a in org_address_elements if "maps" in a["href"]), None)
        # Extract the Google Maps URL
        org_googlemap = org_googlemap_element["href"] if org_googlemap_element else None

        # Extract all <p> tags
        p_tags = partner.find_all("p")

        # Find the <p> tag that contains an address
        org_address = None
        for p in p_tags:
            text = p.get_text(separator=" ", strip=True)
            # A simple heuristic: Addresses usually contain numbers and street names
            if any(char.isdigit() for char in text) and "Tel" not in text:
                org_address = text
                break  # Stop after finding the first valid address

        # Organize extracted data into a dictionary
        item_data = {
            'Organisation Name': org_name,
            'Phone': phone,
            'Languages': languages,
            'Googlemaps Address':org_googlemap,
            'Address': org_address
        }

        # Append the data to the list
        all_page_data.append(item_data)

    # Print or process the collected data
    for data in all_page_data:
        print(data)
    # Close the driver
    driver.quit()

{'Organisation Name': 'ABL  Accès Accueil Action', 'Phone': 'Tel. :  514\xa0570-3313', 'Languages': 'Français, Anglais, Espagnol, Portugais', 'Googlemaps Address': 'https://maps.app.goo.gl/DvhX1rvEzepHEFmw9', 'Address': None}
{'Organisation Name': 'ABL Accès Accueil Action Basses-Laurentides', 'Phone': 'Tel. :  514 570-3313', 'Languages': 'Français, Anglais, Espagnol, Arabe, Berbère, Portugais, Russe, Ukrainien, Roumain, Batoufam, Bankondji', 'Googlemaps Address': 'https://www.google.ca/maps/place/184+Rue+Saint-Eustache,+Saint-Eustache,+QC+J7R+2L7/@45.5582506,-73.8959556,17z/data=!3m1!4b1!4m6!3m5!1s0x4cc9257e00a9ea8b:0xb751762efa731301!8m2!3d45.5582469!4d-73.8937669!16s%2Fg%2F11b8v6l0xx', 'Address': None}
{'Organisation Name': "Accompagnement des Femmes Immigrantes de L'Outaouais (AFIO)", 'Phone': 'Tel. :  819 776-6764', 'Languages': 'Français, Anglais, Arabe, Lingala, Créole, Kikongo, Tamoul, Allemand, Espagnol, Farsi, Swahili', 'Googlemaps Address': 'http://maps.google.ca/maps?q=109,

/var/folders/xq/xdbqztz92zs24q39hvzwj65m0000gn/T/ipykernel_42299/3368822449.py:22: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  phone_element = partner.find(text=lambda text: 'Tel.' in text if text else False)


NameError: name 'driver' is not defined

In [ ]:
# # Initialize Splinter Browser
# with Browser('chrome') as browser:
#     # Define base URL
#     url = 'https://ircc.canada.ca/english/newcomers/services/index.asp'
#     browser.visit(url)
#     html = browser.html
#     soup = BeautifulSoup(html, 'html.parser')

#     # List to store all extracted data
#     all_page_data = []

#     # Find all rows (tr) in the table
#     rows = soup.find_all('tr')

#     # Iterate through each row to extract data
#     for row in rows:
#         # Extract Organisation Name
#         org_name_element = row.find('a', string=True)  # Find the <a> tag containing the org name
#         org_name = org_name_element.text.strip() if org_name_element else None

#         # Extract Organisation Website
#         org_website_element = row.find('a', href=True)
#         org_website = org_website_element['href'] if org_website_element else None
        
#         # Extract all <a> tags with href in the row
#         org_address_elements = row.find_all('a', href=True)

#         # Extract the second <a> tag, assuming it's the Google Maps link
#         org_googlemap_element = org_address_elements[1] if len(org_address_elements) > 1 else None
#         org_googlemap = org_googlemap_element['href'] if org_googlemap_element else None

#         # Extract the second <a> tag in the row (if it exists)
#         org_address_element = org_address_elements[1] if len(org_address_elements) > 1 else None  # Get the second <a> tag
#         org_address = org_address_element.text.strip() if org_address_element else None

#         # Extract Languages of Service
#         lang_element = row.find('td')  # Assume languages are in a <td> tag
#         if lang_element and "Languages of service:" in lang_element.text:
#             org_languages = lang_element.text.split("Languages of service:")[1].strip()
#         else:
#             org_languages = None

#         # Extract Services
#         services_elements = row.find_all('li', class_="nowrap")
#         org_services = [service.text.strip() for service in services_elements]
        
#         # Initialize an empty list to store the texts of all <td> elements
#         all_td_text = []

#         # Find all <td> tags in the row (table data cells)
#         td_elements = row.find_all('td')
        
#         # Iterate through each <td> tag in the row
#         for td in td_elements:
#             # Remove all <a> tags to avoid extracting the hyperlink text
#             for a_tag in td.find_all('a'):
#                 a_tag.decompose()  # This will remove the <a> tag and its content from the <td>
            
#             # Get the text inside the <td> and strip any surrounding whitespace
#             cell_text = td.get_text(strip=True)
            
#             if cell_text:  # Add non-empty text to the list
#                 all_td_text.append(cell_text)

#         # Join the list of texts from all <td> elements into a single string
#         org_details = '/'.join(all_td_text)

#         # Organize extracted data into a dictionary
#         item_data = {
#             'Organisation Name': org_name,
#             'Organisation Website': org_website,
#             'Organisation Address': org_address,
#             'Organisation Languages': org_languages,
#             'Organisation Services': org_services,
#             'Organisation Google Map': org_googlemap,
#             'Organisation Details': org_details  # Store the combined <td> contents
#         }

#         # Append the data to the list
#         all_page_data.append(item_data)

#     # Print or process the collected data
#     for data in all_page_data:
#         print(data)

